In [1]:
from conda_forge_tick.lazy_json_backends import LazyJson, lazy_json_override_backends

In [9]:
!rm -rf node_attrs pr_info pr_json version_pr_info

In [12]:
with lazy_json_override_backends(["github"], use_file_cache=True):
    attrs = LazyJson("node_attrs/ngmix.json")
    print(len(attrs.data))

28


In [8]:
from collections.abc import Collection, Mapping


def _sync_node(data, seen=None):
    seen = seen or []

    if isinstance(data, LazyJson):
        data.data

    if isinstance(data, Mapping):
        for v in data.values():
            if v not in seen:
                seen.append(v)
                seen = _sync_node(v, seen=seen)
    elif (
        isinstance(data, Collection)
        and not isinstance(data, str)
        and not isinstance(data, bytes)
    ):
        for v in data:
            if v not in seen:
                seen.append(v)
                seen = _sync_node(v, seen=seen)

    return seen


with lazy_json_override_backends(["github"]):
    ngmix = LazyJson("node_attrs/ngmix.json")
    _sync_node(ngmix)
    ngmix2 = LazyJson("node_attrs/ngmix.json")
    _sync_node(ngmix2)

    print(ngmix == ngmix2)

    with ngmix["pr_info"] as pri:
        pri.clear()
    print(ngmix == ngmix2)

    del ngmix.data["pr_info"]
    print(ngmix == ngmix2)

True
False
False
